# 한화토탈 8월 24일 ML 실습자료

In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings(action='ignore')

### 0. Titanic 데이터 로드 및 훓어보기

In [2]:
# 여러분이 이전 과제로 만드셨을 FE를 진행한 데이터 보다는 원본 데이터가 조금 더 이번 실습에 적합하기 때문에 원본을 사용하겠습니다.
# 데이터를 로드하고, 잘 불러와진 것인지 첫 5행을 불러와 확인해보세요.

titanic = pd.read_csv('titanic.csv')
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
# 변수들의 데이터 타입, 결측치, target의 imbalance 여부를 확인해주세요.
titanic.info()
titanic.Survived.value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


0    549
1    342
Name: Survived, dtype: int64

In [4]:
# 편의상 결측치 데이터 및 승객 고유의 데이터('PassengerId','Name', 'Ticket', 'Cabin')는 모두 drop 시킵시다.
# 실제 프로젝트에서는 이렇게 하시면 성능이 낮을 수 있어요!!
titanic = titanic.dropna()
titanic = titanic.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis = 1)
titanic

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
1,1,1,female,38.0,1,0,71.2833,C
3,1,1,female,35.0,1,0,53.1000,S
6,0,1,male,54.0,0,0,51.8625,S
10,1,3,female,4.0,1,1,16.7000,S
11,1,1,female,58.0,0,0,26.5500,S
...,...,...,...,...,...,...,...,...
871,1,1,female,47.0,1,1,52.5542,S
872,0,1,male,33.0,0,0,5.0000,S
879,1,1,female,56.0,0,1,83.1583,C
887,1,1,female,19.0,0,0,30.0000,S


In [5]:
# categorical feature(Sex, Embarked)에 대해 LabelEncoding을 진행해 주세요.
# 전처리 완료된 DataFrame을 첫 5행을 불러와 확인해보세요
display(titanic.Sex.value_counts())
display(titanic.Embarked.value_counts())

male      95
female    88
Name: Sex, dtype: int64

S    116
C     65
Q      2
Name: Embarked, dtype: int64

In [6]:
titanic = pd.get_dummies(titanic, columns = ['Sex', 'Embarked'])
titanic.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
1,1,1,38.0,1,0,71.2833,1,0,1,0,0
3,1,1,35.0,1,0,53.1000,1,0,0,0,1
6,0,1,54.0,0,0,51.8625,0,1,0,0,1
10,1,3,4.0,1,1,16.7000,1,0,0,0,1
11,1,1,58.0,0,0,26.5500,1,0,0,0,1


### 1. Train-Test split & 종속변수-설명변수 분할

In [7]:
# 분할을 위해 필요한 적절한 패키지를 불러와주세요 
# train-test split 및 종속변수-설명변수 분할을 진행해주세요
from sklearn.model_selection import train_test_split
X = titanic.drop('Survived', axis = 1)
y = titanic.Survived
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 32)

In [8]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(146, 10)
(37, 10)
(146,)
(37,)


### 2. Decision Tree를 통한 Classification

In [9]:
# Decision Tree에서 categorical feature인 'Pclass'를 기준으로 첫 번째 Node(Root Node)가 분리된다고 가정합시다.
# 이 때 Entropy 지표를 통해 Information Gain을 계산할 수 있는 함수(또는 코드)를 작성하고, Information gain을 계산하세요.
# 실습에서 다루지 않은 내용입니다. 코드가 어려우시다면 줄글로만 풀이과정을 작성해주셔도 됩니다.

def getEntropy(df):
    E = 0
    values = df.Survived.value_counts().tolist()
    for v in values:
        p = v / len(df)
        E -= p * np.log2(p)
    return E    

# Information gain
gain = getEntropy(titanic)
for pclass, val in enumerate(titanic.Pclass.value_counts().tolist()):
    gain -= val / len(titanic) * getEntropy(titanic[titanic.Pclass == (pclass + 1)])

print(gain)

# titanic data로 Pclass를 분리기준으로 이용했을 때의 entropy를 계산하는 상황만 고려하시면 됩니다.
# 계산에 사용할 데이터는 분할 이전의 0번에서 전처리 완료한 데이터를 기준으로 하겠습니다!
# 올바르게 계산할 시 Information Gain은 0.0097이 나옵니다.


0.00976231915019337


##### <b>줄글 답안</b>
>

In [10]:
# Decision Tree 모델을 사용하기 위한 적절한 패키지를 불러와 주세요
# 별 다른 parameter 설정 없이 객체를 만들어 주고, 학습을 진행하세요
from sklearn.tree import DecisionTreeClassifier

dt_clf = DecisionTreeClassifier(criterion = 'entropy')
dt_clf.fit(X_train, y_train)

DecisionTreeClassifier(criterion='entropy')

### 3. Graphviz를 통한 시각화

graphviz 설치를 하지 않으셨거나 오류가 난다면 이 부분은 건너뛰셔도 좋습니다

In [11]:
from sklearn.tree import export_graphviz

export_graphviz(dt_clf ,
                out_file=("titanic_hw.dot"),
                feature_names=X_train.columns,
                class_names=['0','1'],
                rounded=True,
                filled=True)

import graphviz
import os
print(os.pathsep)

os.environ["PATH"] += os.pathsep + 'E:\Anaconda3\envs\AI3\Library\bin\graphviz'

with open("titanic_hw.dot") as f:
    dot_graph = f.read()
    display(graphviz.Source(dot_graph))

;


ExecutableNotFound: failed to execute ['dot', '-Tsvg'], make sure the Graphviz executables are on your systems' PATH

### 4. 결과 확인

In [12]:
# accuracy, f1 score, roc_auc score, confusion matrix, classification report를 모두 불러와주세요
# 불러온 metric(결과 확인 지표)들을 통해 결과를 확인해주세요.
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix, classification_report

print("----------matrix------------")
pred = dt_clf.predict(X_test)
matrix = confusion_matrix(y_test, pred)
display(matrix)

print("----------scores------------")
print('Accuracy:: {}'.format(accuracy_score(y_test, pred)),
      'F1_Score:: {}'.format(f1_score(y_test, pred)),
      'ROC_AUC_Score:: {}'.format(roc_auc_score(y_test, pred)), sep = '\n')

print("----------report------------")
print(classification_report(y_test, pred))


----------matrix------------


array([[13,  4],
       [ 0, 20]], dtype=int64)

----------scores------------
Accuracy:: 0.8918918918918919
F1_Score:: 0.9090909090909091
ROC_AUC_Score:: 0.8823529411764706
----------report------------
              precision    recall  f1-score   support

           0       1.00      0.76      0.87        17
           1       0.83      1.00      0.91        20

    accuracy                           0.89        37
   macro avg       0.92      0.88      0.89        37
weighted avg       0.91      0.89      0.89        37



### 5. GridSearch를 통한 Tree pruning

In [13]:
# GridSearch를 위한 적절한 패키지를 불러와 주세요
# max_leaf_nodes를 제외한 3가지 이상의 parameter에 대한 GridSearch를 진행해주세요. scoring은 원하시는 지표로 하셔도 됩니다.
from sklearn.model_selection import GridSearchCV
grid_dt = DecisionTreeClassifier()

criterion = ['gini', 'entropy']
max_depth = [3, 4, 5, 6, 7]
max_features = ['auto', 'sqrt', 'log2', None]
grid = {
    'criterion' : criterion,
    'max_depth' : max_depth,
    'max_features' : max_features
}

gs = GridSearchCV(estimator = grid_dt, param_grid = grid, scoring = 'roc_auc')
gs.fit(X_train, y_train)

GridSearchCV(estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [3, 4, 5, 6, 7],
                         'max_features': ['auto', 'sqrt', 'log2', None]},
             scoring='roc_auc')

In [14]:
# 최적 parameter 조합 / Validation score / Test score를 확인해 주세요.
print("best params : {}".format(gs.best_params_))
print("best validation score : {}".format(gs.best_score_))
print("test score : {}".format(gs.score(X_test, y_test)))

best params : {'criterion': 'entropy', 'max_depth': 4, 'max_features': None}
best validation score : 0.8150793650793652
test score : 0.8705882352941177


In [ ]:
## GridSearch에 사용하신 parameter는 Decision Tree 모델 학습에서 어떤 역할을 하는지 간단하게 요약해주세요.
## ex) criterion은 decision tree의 분류 기준인 불순도 지표를 정한다.
## hint) Shift+tab

- criterion : 불순도 지표를 정한다. gini일 경우는 gini impurity, entropy일 경우 information gain  
- max_depth : tree의 maximum depth. default는 none  
- max_feature : 최적의 split을 정할 때 고려되는 feature의 개수. default는 None이고, 이때 max_feature = n_features가 된다.

생각보다 GridSearch의 결과가 안 좋게 나와서 이상하다 생각하신 분들도 있을 수 있습니다. GridSearch 특성상 기존에 정해준 parameter 범위 내에서 찾는 것이기 때문에 애초에 적절한 parameter 후보를 찾지 못했다면 성능이 안 좋을 수도 있습니다. 앞으로 여러 프로젝트 진행해보시면서, parameter 찾는 것에 대한 감도 스스로 익혀보시면 좋을 것 같습니다!